# Challenge Kaggle - Visual In-Depth EDA – VinBigData Competition Data

    Neural network 3

### Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import pydicom
import os
import glob
import pickle
import pandas as pd
import csv
import cv2
from pathlib import Path
from IPython.display import HTML
from os import listdir, mkdir
import imageio
from IPython import display
from sklearn.metrics import roc_auc_score, roc_curve, auc
import scipy

import tensorflow as tf
from keras.models import load_model
from keras import layers 
from keras.layers import Concatenate,Input, Dense, Activation, ZeroPadding2D, \
BatchNormalization, Flatten, Conv2D 
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, \
GlobalAveragePooling2D 
from keras.models import Model, Sequential 
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file 
from keras.applications.imagenet_utils import preprocess_input 
from sklearn.metrics import roc_auc_score, roc_curve, auc

### Load data

In [ ]:
Validacao    = 'C:/Users/felipe/Documents/DataBOX/Box_Pulm_val.pckl'
fv = open(Validacao, 'rb')
[Xv,Yv] = pickle.load(fv)

In [ ]:
plt.imshow(Xv[0,:,:,1],cmap=plt.cm.bone)
print(Yv[0])

### Construct the model

In [ ]:
def KerasModel(input_shape):  
    NumClass = 4
    X_input  = Input(input_shape)
     
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(40, (3, 3), strides = (1, 1), name = 'conv0')(X)    
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    ##
    X = Conv2D(24, (3, 3), strides = (1, 1), name = 'conv1')(X)    
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)  
    ##
    
    ##
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv2')(X)    
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)  
    ##    

    ##   New layer
    X = Conv2D(8, (3, 3), strides = (1, 1), name = 'conv3')(X)    
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool3')(X)  
    ##    
    
    ##   New layer
    X = Conv2D(4, (3, 3), strides = (1, 1), name = 'conv4')(X)    
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool4')(X)  
    ##    

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(NumClass, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='KerasModel')
    
    return model 

In [ ]:
# Construct the model
ModelPulm = KerasModel(Xv[0].shape) # input dimention
# Run the model
ModelPulm.compile(loss='binary_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [ ]:
ModelPulm.summary()

In [ ]:
tf.keras.utils.plot_model(
    ModelPulm, to_file='C:/Users/felipe/Documents/Pulm/model.png', show_shapes=True, show_layer_names=True,
    rankdir='TB', expand_nested=True, dpi=96
)

### Run the model

In [ ]:
plot_hist = []
plot_val_hist = []
plot_loss = []
plot_val_loss = []
model = []
acc = []
val_acc = []
for i in range(0,7):
    print('Number of run: ' + str(i))
    save_history = 'C:/Users/felip/Documents/Pulm/history_' + str(i) + '.png'
    save_loss    = 'C:/Users/felip/Documents/Pulm/loss_' + str(i) + '.png'
    save_model   = 'C:/Users/felip/Documents/Pulm/ModelBOX_' + str(i) + '.h5'
    Path         = 'C:/Users/felip/Documents/DataBOX/Box_Pulm' + str(i) + '.pckl'
    f = open(Path, 'rb')
    [Xt,Y] = pickle.load(f)
    
    # save fit model
    history = ModelPulm.fit(x = Xt, y = Y, batch_size = 2**6, epochs = 16, validation_data = (Xv,Yv)) 
    
    #save model
    ModelPulm.save(save_model)
    
    model.append(i)
    acc.append(history.history['accuracy'][15])
    val_acc.append(history.history['val_accuracy'][15])
    plot_hist.extend(history.history['accuracy'])
    plot_val_hist.extend(history.history['val_accuracy'])
    plot_loss.extend(history.history['loss'])
    plot_val_loss.extend(history.history['val_loss'])
    
    # save validation fit model
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(save_history, format='png')
    plt.show()
   
    # save validation loss model
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(save_loss, format='png')
    plt.show()

### Plot training

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(plot_hist)
plt.plot(plot_val_hist)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'loss', 'val_loss'], loc='upper left')
plt.savefig('C:/Users/felip/Documents/Pulm/history-history.png', format='png')
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(plot_hist)
plt.plot(plot_val_hist)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'loss', 'val_loss'], loc='upper left')
plt.savefig('C:/Users/felip/Documents/Pulm/history-history.png', format='png')
plt.show()

In [ ]:
data = {'Model': model,
        'acc': acc,
        'val_acc': val_acc}

df = pd.DataFrame(data, columns = ['Model', 'acc', 'val_acc'])
df.to_csv('C:/Users/felip/Documents/Pulm/acc.csv')
df.head(10)

### Validation

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = ModelPulm.evaluate(Xv, Yv, batch_size = 64)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for samples")
pred = ModelPulm.predict(Xv)
print("predictions shape:", pred.shape)

### Evaluating

In [ ]:
box = pred * 250
ori = np.float32(Yv * 250)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
box_acc = []
for i in range(1024):
    boxA = ori[i]
    boxB = box[i]
    box_acc.append(bb_intersection_over_union(boxA, boxB))
np.mean(box_acc)

### Test

In [ ]:
def draw_img_box(test_img, ori, box):               
    start_point = (ori[2], ori[3]) 
    end_point = (ori[0], ori[1]) 
    color = (0, 0, 255) 
    thickness = 1
    test_img = cv2.rectangle(test_img, start_point, end_point, color, thickness)

    start_point = (box[2], box[3]) 
    end_point = (box[0], box[1]) 
    color = (255, 0, 0) 
    thickness = 1
    test_img = cv2.rectangle(test_img, start_point, end_point, color, thickness)
    
    return test_img

In [ ]:
fig = plt.figure(figsize = (25,25))
for i in range(16):
    fig_pulm = draw_img_box(Xv[i], ori[i], box[i])
    fig.add_subplot(4, 4, i + 1).imshow(fig_pulm)
plt.savefig('C:/Users/felip/Documents/Pulm/drawbox.png', format='png')